In [1]:
#%matplotlib inline
import numpy as np
#import matplotlib.pyplot as plt
from sklearn import cross_validation
import tensorflow as tf
import random

/home/alex/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
# DONT RUN

#trainData=np.genfromtxt('train.csv',delimiter=',')
testData=np.genfromtxt('test.csv',delimiter=',')
#trainingPixmaps=trainData[:,9:438]/255

In [10]:
trainData = trainData.astype(int)
testData = testData.astype(int)

In [13]:
# NEVER RE RUN . SIMPLY LOAD FROM THE FILES AS BELOW
X_train, X_test, y_train, y_test = cross_validation.train_test_split (trainData, trainData[:,1], 
                                                                      test_size=0.25, random_state=0)
np.savetxt('ensemble/X_train.txt', X_train, delimiter=',')
np.savetxt('ensemble/X_test.txt', X_test, delimiter=',') 
np.savetxt('ensemble/y_train.txt', y_train, delimiter=',') 
np.savetxt('ensemble/y_test.txt', y_test, delimiter=',') 

In [2]:
X_train = np.loadtxt('ensemble/X_train.txt', delimiter=',')
X_test = np.loadtxt('ensemble/X_test.txt', delimiter=',')
y_train = np.loadtxt('ensemble/y_train.txt', delimiter=',')
y_test = np.loadtxt('ensemble/y_test.txt', delimiter=',')

In [24]:
## Don't re run, just load from file :)

from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=350)
forest.fit(X_train[:, 9:438], y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=350, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [25]:
RFPred = forest.predict(X_test[:,9:438])

In [27]:
np.savetxt('ensemble/RFFPred', RFPred, delimiter=',')

In [31]:
test_RFPred = forest.predict(testData[:,9:438])

In [32]:
np.savetxt('ensemble/test_RFPred',test_RFPred,delimiter=',')

In [26]:
numCorect = 0
for i in range (0, len(y_test)) :
    if RFPred[i] == y_test[i]:
        numCorect += 1
print(str(float(numCorect) / float(len(y_test))))

0.679102696683


In [3]:
KKNPred = np.loadtxt('ensemble/predCosKNN.txt',delimiter=',')
RFPred = np.loadtxt('ensemble/RFFPred', delimiter=',')
CNNPred = np.loadtxt('ensemble/CNNPred.txt',delimiter=',')

In [4]:
# Calc ind. predictions 
KNNCorrect = 0
RFCorrect = 0
CNNCorrect = 0
for i in range (0, len(X_test)) :
    if KKNPred[i] == y_test[i] :
        KNNCorrect += 1
    if RFPred[i] == y_test[i] :
        RFCorrect += 1
    if CNNPred[i] == y_test[i] :
        CNNCorrect += 1
    
print("KNN got " + str(float(KNNCorrect) / float(len(X_test))))
print("RF got " + str(float(RFCorrect) / float(len(X_test))))
print("CNN got " +  str(float(CNNCorrect) / float(len(X_test))))


KNN got 0.682682364172
RF got 0.679102696683
CNN got 0.679182244849


In [7]:
# Simple ensemble
simpePreds = []
smplCorrect = 0
for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    thePred = 46
    if (singleKNNPred == singleRFPred) :
        thePred = singleKNNPred
    else :
        if singleRFPred == singleCNNPred :
            thePred = singleRFPred
        else :
            # No majority, choose best classifier
            thePred = singleKNNPred
    simpePreds.append(thePred)
    if thePred == y_test[i] :
        smplCorrect += 1
        
print("Simple ensemble got " + str(float(smplCorrect) / float(len(X_test))))

Simple ensemble got 0.690318988147


In [18]:
# Complex ensemble based on correct percentage
KNNpercents = getPercentageCorrect(KKNPred)
RFpercents = getPercentageCorrect(RFPred)
CNNpercents = getPercentageCorrect(CNNPred)

In [17]:
# More complex ensemble
cmplxCorrect = 0
numAllDiff = 0
for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    thePred = 46
    if (singleKNNPred == singleRFPred) :
        thePred = singleKNNPred
    else :
        if singleRFPred == singleCNNPred :
            thePred = singleRFPred
        else :
            numAllDiff += 1
            # No majority, choose most likely one
            KNNperc = KNNpercents[int(singleKNNPred)]
            RFperc = RFpercents[int(singleRFPred)]
            CNNperc = CNNpercents[int(singleCNNPred)]
            if (KNNperc > RFperc and KNNperc > CNNperc) :
                thePred = singleKNNPred #KNN is most likely to be right
            else :
                if (RFperc > CNNperc) :
                    thePred = singleRFPred
                else :
                    thePred = singleCNNPred

    if thePred == y_test[i] :
        cmplxCorrect += 1
        
print(numAllDiff)
print("Complex ensemble got " + str(float(cmplxCorrect) / float(len(X_test))))

829
Complex ensemble got 0.687693898656


In [ ]:
# Below is code that we did experiments with, but did not form part of the final version. 

In [18]:
# Dereks ensemble
derekCorrect = 0
for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    thePred = 46
 
    
    KNNperc = KNNpercents[int(singleKNNPred)]
    RFperc = RFpercents[int(singleRFPred)]
    CNNperc = CNNpercents[int(singleCNNPred)]
    if (KNNperc > RFperc and KNNperc > CNNperc) :
        thePred = singleKNNPred #KNN is most likely to be right
    else :
        if (RFperc > CNNperc) :
            thePred = singleRFPred
        else :
            thePred = singleCNNPred

    if thePred == y_test[i] :
        derekCorrect += 1

        
print("Derek got " + str(float(derekCorrect) / float(len(X_test))))

Derek got 0.683955134834


In [19]:
# Alexs ensemble - class freq
alexCorrect = 0
for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    thePred = 46
    if (singleKNNPred == singleRFPred) :
        thePred = singleKNNPred
    else :
        if singleRFPred == singleCNNPred :
            thePred = singleRFPred
        else :
            # No majority, choose most populat example
            thePred = getMostPopular(singleKNNPred,singleRFPred,singleCNNPred)
    if thePred == y_test[i] :
        alexCorrect += 1

        
print("Alex got " + str(float(alexCorrect) / float(len(X_test))))

Alex got 0.671147880041


In [20]:
# Unless all the same ensemble
uasCorrect = 0
for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    thePred = 46
    if (singleKNNPred == singleRFPred and singleKNNPred == singleCNNPred) :
        thePred = singleKNNPred
    else :
        # No complete majority, choose best classifier
        thePred = singleKNNPred
    if thePred == y_test[i] :
        uasCorrect += 1
        
print("Unless all same ensemble got " + str(float(uasCorrect) / float(len(X_test))))

Unless all same ensemble got 0.682682364172


In [10]:
classFreqs = np.zeros(98)
for i in range (0, len(y_train)):
    classFreqs[int(y_train[i])] += 1

In [17]:
def getMostPopular(class1, class2, class3) :
    class1Freq = classFreqs[int(class1)]
    class2Freq = classFreqs[int(class2)]
    class3Freq = classFreqs[int(class3)]
    if(class1Freq > class2Freq and class1Freq > class3Freq) :
        return class1
    else :
        if class2Freq > class3Freq :
            return class2Freq
        else :
            return class3Freq

In [16]:
def getPercentageCorrect(pred) :
    counts = np.zeros(98)
    countsCorrect = np.zeros(98)
    for i in range (0, len(pred)) :
        thePred = int(pred[i])
        theRealValue = y_test[i]
        counts[thePred] += 1
        if (thePred == theRealValue) :
            countsCorrect[thePred] += 1
    percentages = []
    for i in range(0,98) :
        if(counts[i] == 0) :
            percentages.append(0.0)
        else :
            percentages.append(float(countsCorrect[i]) / float(counts[i]))
    return percentages
        

In [10]:
testRFPred = np.loadtxt('ensemble/test_RFPred',delimiter=',')

In [11]:
testCNNPred = np.loadtxt('ensemble/TestCNNPred.txt',delimiter=',')

In [12]:
testKNNPred = np.loadtxt('ensemble/testKNN.txt',delimiter=',')



In [ ]:
# More complex ensemble
cmplxCorrect = 0
numAllDiff = 0
for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    thePred = 46
    if (singleKNNPred == singleRFPred) :
        thePred = singleKNNPred
    else :
        if singleRFPred == singleCNNPred :
            thePred = singleRFPred
        else :
            numAllDiff += 1
            # No majority, choose most likely one
            KNNperc = KNNpercents[int(singleKNNPred)]
            RFperc = RFpercents[int(singleRFPred)]
            CNNperc = CNNpercents[int(singleCNNPred)]
            if (KNNperc > RFperc and KNNperc > CNNperc) :
                thePred = singleKNNPred #KNN is most likely to be right
            else :
                if (RFperc > CNNperc) :
                    thePred = singleRFPred
                else :
                    thePred = singleCNNPred

    if thePred == y_test[i] :
        cmplxCorrect += 1
        
print(numAllDiff)
print("Complex ensemble got " + str(float(cmplxCorrect) / float(len(X_test))))

In [19]:
# Simple ensemble for testData
smplCorrect = 0
simpleTestPreds = []
for i in range (0, len(testData)) :
    singleKNNPred = testKNNPred[i]
    singleRFPred = testRFPred[i]
    singleCNNPred = testCNNPred[i]
    thePred = 46
    if (singleKNNPred == singleRFPred) :
        thePred = singleKNNPred
    else :
        if singleRFPred == singleCNNPred :
            thePred = singleRFPred
        else :
            # No majority, choose most likely one
            KNNperc = KNNpercents[int(singleKNNPred)]
            RFperc = RFpercents[int(singleRFPred)]
            CNNperc = CNNpercents[int(singleCNNPred)]
            if (KNNperc > RFperc and KNNperc > CNNperc) :
                thePred = singleKNNPred #KNN is most likely to be right
            else :
                if (RFperc > CNNperc) :
                    thePred = singleRFPred
                else :
                    thePred = singleCNNPred
    simpleTestPreds.append(thePred)
        
testIds = testData[:, 0]

resultsForCSV = []
for i in range(0, len(testIds)) :
    temp = []
    theId = int(testIds[i])
    theClass = int(simpleTestPreds[i])
    temp.append(theId)
    temp.append(theClass)
    resultsForCSV.append(temp)

    

In [20]:
import csv

with open("KNNCNNRF_Ensemble_Complex.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(resultsForCSV)

In [23]:
# By Book on tiebreak ensemble
byBookCorrect = 0
for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    thePred = 46
    if (singleKNNPred == singleRFPred) :
        thePred = singleKNNPred
    else :
        if singleRFPred == singleCNNPred :
            thePred = singleRFPred
        else :
            # No majority, choose by the best classifier on the book
            bookNum = int(X_test[i][2])
            byBookKNN = CNNCorrectByBook[bookNum]
            byBookRF = RFCorrectByBook[bookNum]
            byBookCNN = KNNCorrectByBook[bookNum]
            if byBookKNN > byBookRF and byBookKNN > byBookCNN:
                thePred = singleKNNPred
            else :
                if byBookRF > byBookCNN :
                    thePred = singleRFPred
                else :
                    thePred = singleCNNPred
            
    if thePred == y_test[i] :
        byBookCorrect += 1
        
print("By Book ensemble got " + str(float(byBookCorrect) / float(len(X_test))))

By Book ensemble got 0.687693898656


In [ ]:
# Look into accuaracy by book num

In [6]:
def getPredSimple(singleKNNPred,singleRFPred,singleCNNPred):
    thePred = 46
    if (singleKNNPred == singleRFPred) :
        thePred = singleKNNPred
    else :
        if singleRFPred == singleCNNPred :
            thePred = singleRFPred
        else :
            # No majority, choose best classifier
            thePred = singleKNNPred
    return thePred

In [9]:
# Calc ind. predictions 
KNNCorrectByBook = np.zeros(6)
RFCorrectByBook = np.zeros(6)
CNNCorrectByBook = np.zeros(6)
SimpleCorrectByBook = np.zeros(6)
numByBook = np.zeros(6)

for i in range (0, len(X_test)) :
    bookNum = int(X_test[i][2])
    numByBook[bookNum] += 1
    if KKNPred[i] == y_test[i] :
        KNNCorrectByBook[bookNum] += 1
    if RFPred[i] == y_test[i] :
        RFCorrectByBook[bookNum] += 1
    if CNNPred[i] == y_test[i] :
        CNNCorrectByBook[bookNum] += 1
    if simpePreds[i] == y_test[i] :
        SimpleCorrectByBook[bookNum] += 1
for i in range(0, 6):
    KNNCorForBook = KNNCorrectByBook[i]
    RFCorForBook = RFCorrectByBook[i]
    CNNCorForBook = CNNCorrectByBook[i]
    SimpleCorForBook  = SimpleCorrectByBook[i]
    numInBook = numByBook[i]
    print("For book number " + str(i))
    print("KNN got " + str(float(KNNCorForBook) / float(numInBook)))
    print("RF got " + str(float(RFCorForBook) / float(numInBook)))
    print("CNN got " +  str(float(CNNCorForBook) / float(numInBook)))
    print("Simple got " +  str(float(SimpleCorForBook) / float(numInBook)) + "\n")

For book number 0
KNN got 0.43880455408
RF got 0.425996204934
CNN got 0.415559772296
Simple got 0.44165085389

For book number 1
KNN got 0.565656565657
RF got 0.56632996633
CNN got 0.563636363636
Simple got 0.576430976431

For book number 2
KNN got 0.862330623306
RF got 0.862330623306
CNN got 0.866395663957
Simple got 0.868292682927

For book number 3
KNN got 0.774016468435
RF got 0.773101555352
CNN got 0.760292772187
Simple got 0.783165599268

For book number 4
KNN got 0.60412686282
RF got 0.604508979748
CNN got 0.602980512037
Simple got 0.608712265953

For book number 5
KNN got 0.765525982256
RF got 0.753485424588
CNN got 0.772496831432
Simple got 0.7845373891



In [76]:
# Simple ensemble anaysis of what is wrong
smplCorrect = 0
simpleEsemblePred = []
for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    thePred = 46
    if (singleKNNPred == singleRFPred) :
        thePred = singleKNNPred
    else :
        if singleRFPred == singleCNNPred :
            thePred = singleRFPred
        else :
            # No majority, choose best classifier
            thePred = singleKNNPred
    simpleEsemblePred.append(thePred)

In [82]:
predClassWrong = []
realClassWrong = []
predWrongByClass = np.zeros(98)
realWrongByClass = np.zeros(98)

for i in range (0, len(simpleEsemblePred)):
    thePred = simpleEsemblePred[i]
    theReal = y_test[i]
    if (thePred != theReal) :
        predClassWrong.append(thePred)
        realClassWrong.append(theReal)
        predWrongByClass[int(thePred)] += 1
        realWrongByClass[int(theReal)] += 1

In [81]:
from scipy import stats
m = stats.mode(predClassWrong)
print(m)

m2 = stats.mode(realClassWrong)
print(m2)

ModeResult(mode=array([ 72.]), count=array([465]))
ModeResult(mode=array([ 68.]), count=array([333]))


In [90]:
def getCounts(pred) :
    counts = np.zeros(98)

    for i in range (0, len(pred)) :
        thePred = int(pred[i])
        theRealValue = y_test[i]
        counts[thePred] += 1
    return counts

In [93]:
percentages = getPercentageCorrect(simpleEsemblePred)
counts = getCounts(simpleEsemblePred)
for i in range (0,98) :
    if counts[i] > 50 :
        print("For class " + str(i) + " there were " + str(1 - percentages[i])
         + " predicted wrong values with " + str(counts[i]) + " values")

For class 5 there were 0.347826086957 predicted wrong values with 207.0 values
For class 6 there were 0.2875 predicted wrong values with 80.0 values
For class 7 there were 0.29375 predicted wrong values with 160.0 values
For class 64 there were 0.287912087912 predicted wrong values with 1365.0 values
For class 65 there were 0.526315789474 predicted wrong values with 57.0 values
For class 66 there were 0.305007587253 predicted wrong values with 659.0 values
For class 67 there were 0.34756097561 predicted wrong values with 328.0 values
For class 68 there were 0.279069767442 predicted wrong values with 1118.0 values
For class 70 there were 0.212121212121 predicted wrong values with 66.0 values
For class 71 there were 0.320512820513 predicted wrong values with 234.0 values
For class 72 there were 0.2937460518 predicted wrong values with 1583.0 values
For class 74 there were 0.314666666667 predicted wrong values with 750.0 values
For class 75 there were 0.33704735376 predicted wrong values 

In [35]:
# Anaysis
# Simple ensemble
corKKRFCN = 0
corKKRF = 0
corRFCN = 0
corKKCN = 0
wroKKRFCN = 0
wroKKRF = 0 
wroRFCN = 0 
wroKKCN = 0

corKK = 0
corRF = 0
corCN = 0
wroKK = 0
wroRF = 0
wroCN = 0

for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    corPred = y_test[i]
    
    if singleKNNPred == singleRFPred and singleKNNPred == singleCNNPred  : # If all same
        if (corPred == singleKNNPred) :
            corKKRFCN += 1
        else :
            wroKKRFCN += 1
    else :
        if singleKNNPred == singleRFPred :
            if (corPred == singleKNNPred) :
                corKKRF += 1
            else :
                wroKKRF += 1 
        else :
            if singleKNNPred == singleCNNPred :
                if (corPred == singleKNNPred) :
                    corKKCN += 1
                else :
                    wroKKCN += 1 
            else :
                if singleRFPred == singleCNNPred :
                    if (corPred == singleRFPred) :
                        corRFCN += 1
                    else :
                        wroRFCN += 1  
                else :
                    # None the same
                    if singleKNNPred == corPred :
                        corKK+=1
                    else :
                        wroKK += 1
                    if singleRFPred == corPred :
                        corRF += 1
                    else :
                        wroRF += 1
                    if singleCNNPred == corPred :
                        corCN += 1
                    else :
                        wroCN += 1


                        
                        
print("KK RF CN got us " + str(float(corKKRFCN) / float(corKKRFCN + wroKKRFCN) ) + " with " + str(corKKRFCN + wroKKRFCN))
print("KK RF got us " + str(float(corKKRF) / float(corKKRF + wroKKRF) ) + " with " + str(corKKRF + wroKKRF))
print("KK CN got us " + str(float(corKKCN) / float(corKKCN + wroKKCN) ) + " with " + str(corKKCN + wroKKCN))
print("RF CN got us " + str(float(corRFCN) / float(corRFCN + wroRFCN) ) + " with " + str(corRFCN + wroRFCN))
print("KK got us " + str(float(corKK) / float(corKK + wroKK) ) + " with " + str(corKK + wroKK))
print("RF got us " + str(float(corRF) / float(corRF + wroRF) ) + " with " + str(corRF + wroRF))
print("CN got us " + str(float(corRF) / float(corRF + wroRF) ) + " with " + str(corRF + wroRF))



KK RF CN got us 0.752563863509 with 10726
KK RF got us 0.348333333333 with 600
KK CN got us 0.458745874587 with 303
RF CN got us 0.367965367965 with 462
KK got us 0.183333333333 with 480
RF got us 0.114583333333 with 480
CN got us 0.114583333333 with 480


In [ ]:
# So basically, we have this:


In [40]:
# Anaysis
# Simple ensemble
byClassMeh = np.zeros(98)

for i in range (0, len(X_test)) :
    singleKNNPred = KKNPred[i]
    singleRFPred = RFPred[i]
    singleCNNPred = CNNPred[i]
    corPred = y_test[i]
    
    i = 0
    
    if singleKNNPred == singleRFPred and singleKNNPred == singleCNNPred  : # If all same
        i = 1
    else :
        if singleKNNPred == singleRFPred :
            i = 1
        else :
            if singleKNNPred == singleCNNPred :
                i = 1
            else :
                if singleRFPred == singleCNNPred :
                    i = 1
                else :
                    # None the same
                    byClassMeh[int(corPred)] += 1
 



print(sum(byClassMeh))                    
for i in range(0, 98) :
    if byClassMeh[i] > 30 :
        print("Class " + str(i) + " has " + str(byClassMeh[i]) + " examples")


480.0
Class 64 has 32.0 examples
Class 68 has 35.0 examples
Class 82 has 31.0 examples
